# Working with multiple files

On occasion, we will need to combine more than 2 files using some combination of `UNION` and `JOIN`.  In this lecture, we will show a clean approach to scaling up these operations up to any number of files.  In the process, we will

1. Use `list` comprehensions to process and `UNION` many similar files.
2. Use `dict` comprehensions to store and access many tables by name.

In [1]:
import polars as pl

## Store in `dict` or `list`?

* Natural sequence/order? $\rightarrow$ `list`
    *  Example: Lakes data and years are a natural sequence
* Easier to refer by name? $\rightarrow$ `dict`
    * Baseball files have no order and easier to refer to by name

## Example 1 - Read all baseball database using `dict`

**Task:** Create a `dict` of tables for all tables in the Lahman database

#### Step 1 - Use `glob` to find paths for all CSV files

In [4]:
from glob import glob
files = glob('./data/baseball/core/*.csv')
files

['./data/baseball/core/AwardsManagers.csv',
 './data/baseball/core/Managers.csv',
 './data/baseball/core/AwardsPlayers.csv',
 './data/baseball/core/Fielding.csv',
 './data/baseball/core/Salaries.csv',
 './data/baseball/core/Parks.csv',
 './data/baseball/core/Schools.csv',
 './data/baseball/core/People.csv',
 './data/baseball/core/PitchingPost.csv',
 './data/baseball/core/Teams.csv',
 './data/baseball/core/Appearances.csv',
 './data/baseball/core/AwardsSharePlayers.csv',
 './data/baseball/core/TeamsFranchises.csv',
 './data/baseball/core/Batting.csv',
 './data/baseball/core/ManagersHalf.csv',
 './data/baseball/core/FieldingOF.csv',
 './data/baseball/core/Pitching.csv',
 './data/baseball/core/CollegePlaying.csv',
 './data/baseball/core/HomeGames.csv',
 './data/baseball/core/HallOfFame.csv',
 './data/baseball/core/AwardsShareManagers.csv',
 './data/baseball/core/BattingPost.csv',
 './data/baseball/core/TeamsHalf.csv',
 './data/baseball/core/SeriesPost.csv',
 './data/baseball/core/Fielding

#### Step 2 - Make a function to extract the table name

In [8]:
import re
FILE_NAME_RE = re.compile(r'^\./data/baseball/core/([a-zA-Z_]*)\.csv$')
file_name = lambda p: FILE_NAME_RE.match(p).group(1) if FILE_NAME_RE.match(p) else None

[file_name(p) for p in files]

['AwardsManagers',
 'Managers',
 'AwardsPlayers',
 'Fielding',
 'Salaries',
 'Parks',
 'Schools',
 'People',
 'PitchingPost',
 'Teams',
 'Appearances',
 'AwardsSharePlayers',
 'TeamsFranchises',
 'Batting',
 'ManagersHalf',
 'FieldingOF',
 'Pitching',
 'CollegePlaying',
 'HomeGames',
 'HallOfFame',
 'AwardsShareManagers',
 'BattingPost',
 'TeamsHalf',
 'SeriesPost',
 'FieldingPost',
 'AllstarFull',
 'FieldingOFsplit']

#### 4 - Read in the tables.

In [11]:
baseball_db = {file_name(p):pl.read_csv(p, infer_schema_length=10000) for p in files}

baseball_db

{'AwardsManagers': shape: (179, 6)
 ┌───────────┬───────────────────────────┬────────┬──────┬──────┬───────┐
 │ playerID  ┆ awardID                   ┆ yearID ┆ lgID ┆ tie  ┆ notes │
 │ ---       ┆ ---                       ┆ ---    ┆ ---  ┆ ---  ┆ ---   │
 │ str       ┆ str                       ┆ i64    ┆ str  ┆ str  ┆ str   │
 ╞═══════════╪═══════════════════════════╪════════╪══════╪══════╪═══════╡
 │ larusto01 ┆ BBWAA Manager of the Year ┆ 1983   ┆ AL   ┆ null ┆ null  │
 │ lasorto01 ┆ BBWAA Manager of the Year ┆ 1983   ┆ NL   ┆ null ┆ null  │
 │ andersp01 ┆ BBWAA Manager of the Year ┆ 1984   ┆ AL   ┆ null ┆ null  │
 │ freyji99  ┆ BBWAA Manager of the Year ┆ 1984   ┆ NL   ┆ null ┆ null  │
 │ coxbo01   ┆ BBWAA Manager of the Year ┆ 1985   ┆ AL   ┆ null ┆ null  │
 │ …         ┆ …                         ┆ …      ┆ …    ┆ …    ┆ …     │
 │ willima04 ┆ BBWAA Manager of the Year ┆ 2014   ┆ NL   ┆ null ┆ null  │
 │ banisje01 ┆ BBWAA Manager of the Year ┆ 2015   ┆ AL   ┆ null ┆ null  │
 │ 

### We can now access all the tables by name.

In [12]:
# Biggish output
baseball_db

{'AwardsManagers': shape: (179, 6)
 ┌───────────┬───────────────────────────┬────────┬──────┬──────┬───────┐
 │ playerID  ┆ awardID                   ┆ yearID ┆ lgID ┆ tie  ┆ notes │
 │ ---       ┆ ---                       ┆ ---    ┆ ---  ┆ ---  ┆ ---   │
 │ str       ┆ str                       ┆ i64    ┆ str  ┆ str  ┆ str   │
 ╞═══════════╪═══════════════════════════╪════════╪══════╪══════╪═══════╡
 │ larusto01 ┆ BBWAA Manager of the Year ┆ 1983   ┆ AL   ┆ null ┆ null  │
 │ lasorto01 ┆ BBWAA Manager of the Year ┆ 1983   ┆ NL   ┆ null ┆ null  │
 │ andersp01 ┆ BBWAA Manager of the Year ┆ 1984   ┆ AL   ┆ null ┆ null  │
 │ freyji99  ┆ BBWAA Manager of the Year ┆ 1984   ┆ NL   ┆ null ┆ null  │
 │ coxbo01   ┆ BBWAA Manager of the Year ┆ 1985   ┆ AL   ┆ null ┆ null  │
 │ …         ┆ …                         ┆ …      ┆ …    ┆ …    ┆ …     │
 │ willima04 ┆ BBWAA Manager of the Year ┆ 2014   ┆ NL   ┆ null ┆ null  │
 │ banisje01 ┆ BBWAA Manager of the Year ┆ 2015   ┆ AL   ┆ null ┆ null  │
 │ 

In [13]:
baseball_db['Teams']

yearID,lgID,teamID,franchID,divID,Rank,G,Ghome,W,L,DivWin,WCWin,LgWin,WSWin,R,AB,H,2B,3B,HR,BB,SO,SB,CS,HBP,SF,RA,ER,ERA,CG,SHO,SV,IPouts,HA,HRA,BBA,SOA,E,DP,FP,name,park,attendance,BPF,PPF,teamIDBR,teamIDlahman45,teamIDretro
i64,str,str,str,str,i64,i64,i64,i64,i64,str,str,str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,str,str,i64,i64,i64,str,str,str
1871,"""NA""","""BS1""","""BNA""",null,3,31,null,20,10,null,null,"""N""",null,401,1372,426,70,37,3,60,19,73,16,null,null,303,109,3.55,22,1,3,828,367,2,42,23,243,24,0.834,"""Boston Red Stockings""","""South End Grounds I""",null,103,98,"""BOS""","""BS1""","""BS1"""
1871,"""NA""","""CH1""","""CNA""",null,2,28,null,19,9,null,null,"""N""",null,302,1196,323,52,21,10,60,22,69,21,null,null,241,77,2.76,25,0,1,753,308,6,28,22,229,16,0.829,"""Chicago White Stockings""","""Union Base-Ball Grounds""",null,104,102,"""CHI""","""CH1""","""CH1"""
1871,"""NA""","""CL1""","""CFC""",null,8,29,null,10,19,null,null,"""N""",null,249,1186,328,35,40,7,26,25,18,8,null,null,341,116,4.11,23,0,0,762,346,13,53,34,234,15,0.818,"""Cleveland Forest Citys""","""National Association Grounds""",null,96,100,"""CLE""","""CL1""","""CL1"""
1871,"""NA""","""FW1""","""KEK""",null,7,19,null,7,12,null,null,"""N""",null,137,746,178,19,8,2,33,9,16,4,null,null,243,97,5.17,19,1,0,507,261,5,21,17,163,8,0.803,"""Fort Wayne Kekiongas""","""Hamilton Field""",null,101,107,"""KEK""","""FW1""","""FW1"""
1871,"""NA""","""NY2""","""NNA""",null,5,33,null,16,17,null,null,"""N""",null,302,1404,403,43,21,1,33,15,46,15,null,null,313,121,3.72,32,1,0,879,373,7,42,22,235,14,0.84,"""New York Mutuals""","""Union Grounds (Brooklyn)""",null,90,88,"""NYU""","""NY2""","""NY2"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2019,"""NL""","""SLN""","""STL""","""C""",1,162,81,91,71,"""Y""","""N""","""N""","""N""",764,5449,1336,246,24,210,561,1420,116,29,76,39,662,609,3.8,1,14,52,4332,1284,191,545,1399,66,168,0.989,"""St. Louis Cardinals""","""Busch Stadium III""",3480393,98,97,"""STL""","""SLN""","""SLN"""
2019,"""AL""","""TBA""","""TBD""","""E""",2,162,81,96,66,"""N""","""Y""","""N""","""N""",769,5628,1427,291,29,217,542,1493,94,37,73,34,656,598,3.65,0,12,46,4423,1274,181,453,1621,87,126,0.985,"""Tampa Bay Rays""","""Tropicana Field""",1178735,97,96,"""TBR""","""TBA""","""TBA"""
2019,"""AL""","""TEX""","""TEX""","""W""",3,162,81,78,84,"""N""","""N""","""N""","""N""",810,5540,1374,296,24,223,534,1578,131,38,67,44,878,808,5.06,4,9,33,4314,1515,241,583,1379,105,143,0.982,"""Texas Rangers""","""Globe Life Park in Arlington""",2132994,111,112,"""TEX""","""TEX""","""TEX"""


In [14]:
baseball_db['Batting']

playerID,yearID,stint,teamID,lgID,G,AB,R,H,2B,3B,HR,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP
str,i64,i64,str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,str,i64,i64,str,i64
"""abercda01""",1871,1,"""TRO""","""NA""",1,4,0,0,0,0,0,0,0,0,0,0,null,null,null,null,0
"""addybo01""",1871,1,"""RC1""","""NA""",25,118,30,32,6,0,0,13,8,1,4,0,null,null,null,null,0
"""allisar01""",1871,1,"""CL1""","""NA""",29,137,28,40,4,5,0,19,3,1,2,5,null,null,null,null,1
"""allisdo01""",1871,1,"""WS3""","""NA""",27,133,28,44,10,2,2,27,1,1,0,2,null,null,null,null,0
"""ansonca01""",1871,1,"""RC1""","""NA""",25,120,29,39,11,3,0,16,6,2,2,1,null,null,null,null,0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""zimmejo02""",2019,1,"""DET""","""AL""",23,2,0,0,0,0,0,0,0,0,0,2,"""0""",0,0,"""0""",0
"""zimmeky01""",2019,1,"""KCA""","""AL""",15,0,0,0,0,0,0,0,0,0,0,0,"""0""",0,0,"""0""",0
"""zimmery01""",2019,1,"""WAS""","""NL""",52,171,20,44,9,0,6,27,0,0,17,39,"""0""",0,0,"""2""",4


## Example 2 - Reading and joining the baseball database using `dict`

**Task:** Collect the number of total hits for each batters in the 2010 season join on their first and last name.

In the second example, we will store the data frames in a `dict`, which will make it easier to join the files by ne

#### Step 1 - Get the files names

* Only need the `Batting.csv` and `People.csv`.  
* Narrow with a RegEx

In [34]:
from glob import glob
files = glob('./data/baseball/core/*.csv')
files

['./data/baseball/core/AwardsManagers.csv',
 './data/baseball/core/Managers.csv',
 './data/baseball/core/AwardsPlayers.csv',
 './data/baseball/core/Fielding.csv',
 './data/baseball/core/Salaries.csv',
 './data/baseball/core/Parks.csv',
 './data/baseball/core/Schools.csv',
 './data/baseball/core/People.csv',
 './data/baseball/core/PitchingPost.csv',
 './data/baseball/core/Teams.csv',
 './data/baseball/core/Appearances.csv',
 './data/baseball/core/AwardsSharePlayers.csv',
 './data/baseball/core/TeamsFranchises.csv',
 './data/baseball/core/Batting.csv',
 './data/baseball/core/ManagersHalf.csv',
 './data/baseball/core/FieldingOF.csv',
 './data/baseball/core/Pitching.csv',
 './data/baseball/core/CollegePlaying.csv',
 './data/baseball/core/HomeGames.csv',
 './data/baseball/core/HallOfFame.csv',
 './data/baseball/core/AwardsShareManagers.csv',
 './data/baseball/core/BattingPost.csv',
 './data/baseball/core/TeamsHalf.csv',
 './data/baseball/core/SeriesPost.csv',
 './data/baseball/core/Fielding

#### Step 2 - Make helper functions to get the name from path

In [37]:
import re
FILE_NAME_RE = re.compile(r'^\./data/baseball/core/(Batting|People)\.csv$')
is_batting_or_people = lambda p: FILE_NAME_RE.match(p)
file_name = lambda p: FILE_NAME_RE.match(p).group(1) 

[file_name(p) for p in files if is_batting_or_teams(p)]

['People', 'Batting']

In [39]:
dfs = [pl.read_csv(p) for p in files if is_batting_or_people(p)]

dfs

[shape: (20_090, 24)
 ┌───────────┬───────────┬────────────┬──────────┬───┬───────────┬───────────┬──────────┬───────────┐
 │ playerID  ┆ birthYear ┆ birthMonth ┆ birthDay ┆ … ┆ debut     ┆ finalGame ┆ retroID  ┆ bbrefID   │
 │ ---       ┆ ---       ┆ ---        ┆ ---      ┆   ┆ ---       ┆ ---       ┆ ---      ┆ ---       │
 │ str       ┆ i64       ┆ i64        ┆ i64      ┆   ┆ str       ┆ str       ┆ str      ┆ str       │
 ╞═══════════╪═══════════╪════════════╪══════════╪═══╪═══════════╪═══════════╪══════════╪═══════════╡
 │ aardsda01 ┆ 1981      ┆ 12         ┆ 27       ┆ … ┆ 2004-04-0 ┆ 2015-08-2 ┆ aardd001 ┆ aardsda01 │
 │           ┆           ┆            ┆          ┆   ┆ 6         ┆ 3         ┆          ┆           │
 │ aaronha01 ┆ 1934      ┆ 2          ┆ 5        ┆ … ┆ 1954-04-1 ┆ 1976-10-0 ┆ aaroh101 ┆ aaronha01 │
 │           ┆           ┆            ┆          ┆   ┆ 3         ┆ 3         ┆          ┆           │
 │ aaronto01 ┆ 1939      ┆ 8          ┆ 5        ┆ … ┆ 1962-0

#### Step 3 - Use a comprehension to read in all files

**Note:** The data is small (< 10mb total) so it is safe to read all at once.

In [41]:
dfs = {file_name(p):pl.read_csv(p) for p in files if is_batting_or_teams(p)}
dfs['Batting'].head()

playerID,yearID,stint,teamID,lgID,G,AB,R,H,2B,3B,HR,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP
str,i64,i64,str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,str,str,str,str,i64
"""abercda01""",1871,1,"""TRO""","""NA""",1,4,0,0,0,0,0,0,0,0,0,0,null,null,null,null,0
"""addybo01""",1871,1,"""RC1""","""NA""",25,118,30,32,6,0,0,13,8,1,4,0,null,null,null,null,0
"""allisar01""",1871,1,"""CL1""","""NA""",29,137,28,40,4,5,0,19,3,1,2,5,null,null,null,null,1
"""allisdo01""",1871,1,"""WS3""","""NA""",27,133,28,44,10,2,2,27,1,1,0,2,null,null,null,null,0
"""ansonca01""",1871,1,"""RC1""","""NA""",25,120,29,39,11,3,0,16,6,2,2,1,null,null,null,null,0


In [42]:
dfs['People'].head()

playerID,birthYear,birthMonth,birthDay,birthCountry,birthState,birthCity,deathYear,deathMonth,deathDay,deathCountry,deathState,deathCity,nameFirst,nameLast,nameGiven,weight,height,bats,throws,debut,finalGame,retroID,bbrefID
str,i64,i64,i64,str,str,str,i64,i64,i64,str,str,str,str,str,str,i64,i64,str,str,str,str,str,str
"""aardsda01""",1981,12,27,"""USA""","""CO""","""Denver""",null,null,null,null,null,null,"""David""","""Aardsma""","""David Allan""",215,75,"""R""","""R""","""2004-04-06""","""2015-08-23""","""aardd001""","""aardsda01"""
"""aaronha01""",1934,2,5,"""USA""","""AL""","""Mobile""",null,null,null,null,null,null,"""Hank""","""Aaron""","""Henry Louis""",180,72,"""R""","""R""","""1954-04-13""","""1976-10-03""","""aaroh101""","""aaronha01"""
"""aaronto01""",1939,8,5,"""USA""","""AL""","""Mobile""",1984,8,16,"""USA""","""GA""","""Atlanta""","""Tommie""","""Aaron""","""Tommie Lee""",190,75,"""R""","""R""","""1962-04-10""","""1971-09-26""","""aarot101""","""aaronto01"""
"""aasedo01""",1954,9,8,"""USA""","""CA""","""Orange""",null,null,null,null,null,null,"""Don""","""Aase""","""Donald William""",190,75,"""R""","""R""","""1977-07-26""","""1990-10-03""","""aased001""","""aasedo01"""
"""abadan01""",1972,8,25,"""USA""","""FL""","""Palm Beach""",null,null,null,null,null,null,"""Andy""","""Abad""","""Fausto Andres""",184,73,"""L""","""L""","""2001-09-10""","""2006-04-13""","""abada001""","""abadan01"""


#### Step 4 - Preprocess each file.

In [43]:
# Filter, select, and aggregate hits for 2010.
hits_in_2010_raw = (dfs['Batting']
                   .select(['yearID', 'playerID', 'H'])
                   .filter(pl.col('yearID') == 2010)
                   .group_by('playerID')
                   .agg(pl.col('H').mean().alias('Total Hits'))
                   )
hits_in_2010_raw.head(2)

playerID,Total Hits
str,f64
"""delanro01""",0.0
"""donaljo02""",5.0


In [44]:
# Grab the first and last names from People.

player_names = (dfs['People']
                .select(['playerID', 'nameFirst', 'nameLast'])
               )
player_names.head(2)

playerID,nameFirst,nameLast
str,str,str
"""aardsda01""","""David""","""Aardsma"""
"""aaronha01""","""Hank""","""Aaron"""


#### Step 4 -- Join the tables

In [45]:
hits_in_2010 = (hits_in_2010_raw 
                .join(player_names, on='playerID', how='left')
                .drop('playerID')
               )
hits_in_2010.head()

Total Hits,nameFirst,nameLast
f64,str,str
0.0,"""Rob""","""Delaney"""
5.0,"""Josh""","""Donaldson"""
79.0,"""Miguel""","""Montero"""
12.0,"""Cory""","""Sullivan"""
1.0,"""Mike""","""Baxter"""


## <font color="red"> Exercise 3.2 </font>

We want to get the total hits allowed for all pitchers during the 2000-2010 seasons.  Use `glob` and a `dict` to collect this information into a table that includes the players first and last names.

In [77]:
# Your code here